In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import classification_report

In [2]:
fires = pd.read_csv('4Gridded_Small_Fires.csv')
#Small dataset with tempFires appaerntly worse off

fires = fires.drop(['Unnamed: 0', 'Datetime', 'YEAR', 'MO', 'DY'], axis = 1)
print(fires.head())
print(fires.columns)

   ALLSKY_SFC_LW_DWN  PRECTOT     PS   QV2M  RH2M  SRF_ALB  T10M  T10M_MAX  \
0               5.87     3.13  19.25  10.63  4.32    28.10  0.49      2.06   
1               1.38     1.81  28.12  11.24  0.98    18.91  0.00      0.46   
2               3.09     4.40  22.81  17.17  2.47    32.38  1.65      2.57   
3               3.05     5.91  89.62   4.05  2.06    16.52  3.51      4.33   
4               3.32     1.95  30.00   7.81  0.95    15.08  0.20      0.63   

   T10M_MIN  T10M_RANGE  ...  TROPQ  TROPT  TOA_SW_DNI  TOA_SW_DWN  \
0      0.88        4.81  ...   0.01 -68.68       17.52        8.81   
1      0.93        0.98  ...   0.01 -61.99       17.45        8.71   
2      3.45        4.12  ...   0.01 -67.51       17.43        9.15   
3      4.73        5.57  ...   0.01 -70.45       18.62       10.02   
4      0.30        1.15  ...   0.02 -58.15       16.20        8.23   

   CLRSKY_SFC_PAR_TOT  EVPTRNS  PBLTOP  Longitude  Latitude  Fire  
0              116.92      0.0   81.98   -

In [4]:
from sklearn.preprocessing import StandardScaler
standardizer = StandardScaler()

x = fires.drop(['Fire'], axis = 1)
y = fires['Fire']

x = standardizer.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(10247, 58)
(2562, 58)
(10247,)
(2562,)


In [7]:


model = RandomForestClassifier(n_estimators=10000)
model.fit(x_train, y_train)

print(model.score(x_test, y_test))

0.9652615144418423


In [18]:
from sklearn.inspection import permutation_importance



ValueError: Only integers accepted as `n` values

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
import random

#Randomized Hill Climbing Subset-Select Algorithm
#Feature Selection based on Randomized Hill-Climbing Heuristic

#Global Variables
x_RHCSS = x
y_RHCSS = y
sz = x.shape[1]

x_train, x_test, y_train, y_test = train_test_split(x_RHCSS, y_RHCSS, test_size=0.2, random_state=69)

#Alpha Level, [0, 0.5)
alpha = 0.5
#a% to switch
#100-a% to remain

evaluation_function = accuracy_score
# model_algorithm = RandomForestClassifier
model_algorithm = DecisionTreeClassifier

def score(a, b):
    if len(a) == 0:
        #If we have no features (bitstring = 0), the score is automatically 0
        return 0
    
    model = model_algorithm()
    
    model.fit(a, y_train)
    
    pred = model.predict(b)
    #PREDICTION, Y_TEST OR OTHER WAY AROUND?
    return evaluation_function(pred, y_test)
    
#Generation - Depth of the selection tree
#We select the best children from each generation and continue with it

def RHCSS(generations=100, verbose = 1):
    #Keep running track of the best state we found so far
    #We use a bitset to represent wether the each feature is included
    best = (0, [0] * sz)
    
    for i in range(generations):
        current_state = best[1]
        
        #The features to exclude
        drops = []
        
        #Each generation, we mutate the best state
        for j in range(sz):
            #Probability of each bit flips
            current_state[j] ^= random.random() < alpha
            
            if current_state[j] == 0:
                #If the current bit is excluded, we add it to features to be excluded
                drops.append(j)
        
        #Find score for the current state

        cur_score = score(
            np.delete(x_train, drops, 1),
            np.delete(x_test, drops, 1),
        )
        
        if cur_score > best[0]:
            if verbose == 1:
                print("Better subset found on generation " + str(i) + " with score of " + str(cur_score))
            best = (cur_score, current_state)
        
        if i % 25 == 0 and verbose == 2:
            print("Generation " + str(i) + " completed")
            
            
    return best

res = RHCSS()

Better subset found on generation 0 with score of 0.9352068696330992
Better subset found on generation 42 with score of 0.9395003903200625


In [ ]:
testing_scores = []
testing_bitsets = []

for i in range(100):
    t = RHCSS()
    testing_scores.append(t[0])
    testing_bitsets.append(t[1])
    print(i)

In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
import random

#Randomized Hill Climbing Subset-Select Algorithm
#Feature Selection based on Randomized Hill-Climbing Heuristic

#Global Variables
x_RHCSS = x
y_RHCSS = y
sz = x.shape[1]

x_train, x_test, y_train, y_test = train_test_split(x_RHCSS, y_RHCSS, test_size=0.2, random_state=69)

#Alpha Level, [0, 0.5)
alpha = 0.3
#50% - a% to switch
#50% + a% to remain

evaluation_function = accuracy_score
# model_algorithm = RandomForestClassifier
model_algorithm = DecisionTreeClassifier

def score(a, b):
    if len(a) == 0:
        #If we have no features (bitstring = 0), the score is automatically 0
        return 0
    
    model = model_algorithm()
    
    model.fit(a, y_train)
    
    pred = model.predict(b)
    #PREDICTION, Y_TEST OR OTHER WAY AROUND?
    return evaluation_function(pred, y_test)
    
#Generation - Depth of the selection tree
#We select the best children from each generation and continue with it

def RHCSS(generations=1000, verbose = 1):
    #Keep running track of the best state we found so far
    #We use a bitset to represent wether the each feature is included
    best = (0, [0] * sz)
    
    for i in range(generations):
        genbest = best
        
        for k in range(50):
            current_state = genbest[1]
        
            #The features to exclude
            drops = []
            for j in range(sz):
                #Probability of each bit flips
                current_state[j] ^= random.random() < (0.5 - alpha)

                if current_state[j] == 0:
                    #If the current bit is excluded, we add it to features to be excluded
                    drops.append(j)

            #Find score for the current state

            cur_score = score(
                np.delete(x_train, drops, 1),
                np.delete(x_test, drops, 1),
            )

            if cur_score > best[0]:
                if verbose == 1:
                    print("Better subset found on generation " + str(i) + " with score of " + str(cur_score))
                best = (cur_score, current_state)
        
        if i % 25 == 0 and verbose == 2:
            print("Generation " + str(i) + " completed")
            
            
    return best

res = RHCSS()

Better subset found on generation 0 with score of 0.8657298985167837
Better subset found on generation 0 with score of 0.8946135831381733
Better subset found on generation 0 with score of 0.914519906323185
Better subset found on generation 0 with score of 0.9215456674473068
Better subset found on generation 0 with score of 0.9270101483216238
Better subset found on generation 0 with score of 0.9336455893832943
Better subset found on generation 0 with score of 0.9375487900078064
Better subset found on generation 1 with score of 0.9430132708821234
Better subset found on generation 11 with score of 0.9437939110070258
Better subset found on generation 17 with score of 0.9445745511319282
Better subset found on generation 26 with score of 0.9457455113192819
Better subset found on generation 115 with score of 0.9480874316939891


KeyboardInterrupt: 